In [ ]:
%pip install --upgrade --quiet  langchain kuzu langchain-community langchain-google-genai neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00


In [ ]:
import kuzu

db = kuzu.Database("test_db")
conn = kuzu.Connection(db)

In [ ]:
conn.execute("CREATE NODE TABLE Movie (name STRING, PRIMARY KEY(name))")
conn.execute(
    "CREATE NODE TABLE Person (name STRING, birthDate STRING, PRIMARY KEY(name))"
)
conn.execute("CREATE REL TABLE ActedIn (FROM Person TO Movie)")
conn.execute("CREATE (:Person {name: 'Al Pacino', birthDate: '1940-04-25'})")
conn.execute("CREATE (:Person {name: 'Robert De Niro', birthDate: '1943-08-17'})")
conn.execute("CREATE (:Movie {name: 'The Godfather'})")
conn.execute("CREATE (:Movie {name: 'The Godfather: Part II'})")
conn.execute(
    "CREATE (:Movie {name: 'The Godfather Coda: The Death of Michael Corleone'})"
)
conn.execute(
    "MATCH (p:Person), (m:Movie) WHERE p.name = 'Al Pacino' AND m.name = 'The Godfather' CREATE (p)-[:ActedIn]->(m)"
)
conn.execute(
    "MATCH (p:Person), (m:Movie) WHERE p.name = 'Al Pacino' AND m.name = 'The Godfather: Part II' CREATE (p)-[:ActedIn]->(m)"
)
conn.execute(
    "MATCH (p:Person), (m:Movie) WHERE p.name = 'Al Pacino' AND m.name = 'The Godfather Coda: The Death of Michael Corleone' CREATE (p)-[:ActedIn]->(m)"
)
conn.execute(
    "MATCH (p:Person), (m:Movie) WHERE p.name = 'Robert De Niro' AND m.name = 'The Godfather: Part II' CREATE (p)-[:ActedIn]->(m)"
)

In [ ]:
from langchain.chains import KuzuQAChain
from langchain_community.graphs import KuzuGraph
from langchain_google_genai import GoogleGenerativeAI

In [ ]:
   !pip list | grep langchain-google-genai

In [ ]:
   !pip install --upgrade langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.8 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
from langchain_google_genai import GoogleGenerativeAI

api_key = userdata.get("GOOGLE_API_KEY")
llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=api_key)
print(
  llm.invoke(
      "What are some of the pros and cons of Python as a programming language?"
  )
)

**Pros:**

* **Versatile:** Python is a general-purpose language that can be used for a wide variety of tasks, including web development, data analysis, machine learning, and scripting.
* **Easy to learn:** Python has a simple and straightforward syntax, making it easy to learn for beginners.
* **Large community:** Python has a large and active community, which means that there is a wealth of resources available online, including documentation, tutorials, and forums.
* **Extensive library support:** Python has a large number of libraries available, which can be used to extend the functionality of the language.
* **Open source:** Python is open source, which means that it is free to use and modify.

**Cons:**

* **Slow:** Python is a dynamically typed language, which means that it does not check the types of variables until runtime. This can make Python slower than statically typed languages, such as Java or C++.
* **Memory intensive:** Python is a memory-intensive language, which means

In [ ]:
graph = KuzuGraph(db)
chain = KuzuQAChain.from_llm(llm, graph=graph, verbose=True, allow_dangerous_requests=True)

In [ ]:
chain = KuzuQAChain.from_llm(
  llm,
  graph=graph,
  verbose=True,
  allow_dangerous_requests=True,
  top_k=10  # Limit results
)



In [ ]:
try:
  result = conn.execute("""
      MATCH (p:Person)-[:ActedIn]->(m:Movie)
      WHERE p.name = 'Al Pacino'
      RETURN m.name;
  """)
  print("Direct query results:")
  for row in result:
      print(row)
except Exception as e:
  print("Error in direct query:", e)

Error in direct query: Binder exception: Table Person does not exist.


In [ ]:
chain = KuzuQAChain.from_llm(
  llm,
  graph=graph,
  verbose=True,
  allow_dangerous_requests=True,
  custom_query_template="""
  MATCH (p:Person)-[:ActedIn]->(m:Movie)
  WHERE p.name = '{person_name}'
  RETURN m.name;
  """
)

In [ ]:
try:
  response = chain.run("What movies did Al Pacino act in?")
  print("\nChain response:")
  print(response)
except Exception as e:
  print("Error in chain:", e)



> Entering new KuzuQAChain chain...
Generated Cypher:
()-[:ACTED_IN]->(movie) WHERE movie.name = "Al Pacino"
Error in chain: Parser exception: mismatched input '(' expecting {ALTER, ATTACH, BEGIN, CALL, CHECKPOINT, COMMENT, COMMIT, COPY, CREATE, DELETE, DETACH, DROP, EXPLAIN, EXPORT, IMPORT, INSTALL, LOAD, MATCH, MERGE, OPTIONAL, PROFILE, PROJECT, RETURN, ROLLBACK, SET, UNWIND, USE, WITH, SP} (line: 1, offset: 0)
"()-[:ACTED_IN]->(movie) WHERE movie.name = "Al Pacino""
 ^


In [ ]:
# Let's try a simple MATCH query to see all Person nodes
try:
  result = conn.execute("""
      MATCH (p:Person)
      RETURN p.name, p.birthDate
  """)
  print("People in database:")
  for row in result:
      print(row)
except Exception as e:
  print("Error querying persons:", e)

# Let's see all Movies
try:
  result = conn.execute("""
      MATCH (m:Movie)
      RETURN m.name
  """)
  print("\nMovies in database:")
  for row in result:
      print(row)
except Exception as e:
  print("Error querying movies:", e)

# Let's see all ActedIn relationships
try:
  result = conn.execute("""
      MATCH (p:Person)-[:ActedIn]->(m:Movie)
      RETURN p.name, m.name
  """)
  print("\nActing relationships:")
  for row in result:
      print(row)
except Exception as e:
  print("Error querying relationships:", e)

Error querying persons: Binder exception: Table Person does not exist.
Error querying movies: Binder exception: Table Movie does not exist.
Error querying relationships: Binder exception: Table Person does not exist.


In [ ]:
# First, let's create the database and connection again
db = kuzu.Database("test_db")
conn = kuzu.Connection(db)

# Create the schema
try:
  # Create Person table
  conn.execute("CREATE NODE TABLE Person (name STRING, birthDate STRING, PRIMARY KEY(name))")
  print("Person table created successfully")

  # Create Movie table
  conn.execute("CREATE NODE TABLE Movie (name STRING, PRIMARY KEY(name))")
  print("Movie table created successfully")

  # Create ActedIn relationship
  conn.execute("CREATE REL TABLE ActedIn (FROM Person TO Movie)")
  print("ActedIn relationship created successfully")

  # Insert People
  conn.execute("CREATE (:Person {name: 'Al Pacino', birthDate: '1940-04-25'})")
  conn.execute("CREATE (:Person {name: 'Robert De Niro', birthDate: '1943-08-17'})")
  print("People inserted successfully")

  # Insert Movies
  conn.execute("CREATE (:Movie {name: 'The Godfather'})")
  conn.execute("CREATE (:Movie {name: 'The Godfather: Part II'})")
  conn.execute("CREATE (:Movie {name: 'The Godfather Coda: The Death of Michael Corleone'})")
  print("Movies inserted successfully")

  # Create relationships
  conn.execute("""
      MATCH (p:Person), (m:Movie)
      WHERE p.name = 'Al Pacino' AND m.name = 'The Godfather'
      CREATE (p)-[:ActedIn]->(m)
  """)
  conn.execute("""
      MATCH (p:Person), (m:Movie)
      WHERE p.name = 'Al Pacino' AND m.name = 'The Godfather: Part II'
      CREATE (p)-[:ActedIn]->(m)
  """)
  conn.execute("""
      MATCH (p:Person), (m:Movie)
      WHERE p.name = 'Al Pacino' AND m.name = 'The Godfather Coda: The Death of Michael Corleone'
      CREATE (p)-[:ActedIn]->(m)
  """)
  conn.execute("""
      MATCH (p:Person), (m:Movie)
      WHERE p.name = 'Robert De Niro' AND m.name = 'The Godfather: Part II'
      CREATE (p)-[:ActedIn]->(m)
  """)
  print("Relationships created successfully")

  # Verify the data
  print("\nVerifying data:")
  result = conn.execute("MATCH (p:Person) RETURN p.name, p.birthDate")
  print("\nPeople in database:")
  for row in result:
      print(row)

  result = conn.execute("MATCH (m:Movie) RETURN m.name")
  print("\nMovies in database:")
  for row in result:
      print(row)

  result = conn.execute("MATCH (p:Person)-[:ActedIn]->(m:Movie) RETURN p.name, m.name")
  print("\nActing relationships:")
  for row in result:
      print(row)

except Exception as e:
  print(f"Error occurred: {e}")

Person table created successfully
Movie table created successfully
ActedIn relationship created successfully
People inserted successfully
Movies inserted successfully
Relationships created successfully

Verifying data:

People in database:
Error occurred: 'QueryResult' object is not iterable


In [ ]:
# Verify the data with proper result handling
print("\nVerifying data:")

try:
  # Check People
  print("\nPeople in database:")
  result = conn.execute("MATCH (p:Person) RETURN p.name, p.birthDate")
  while result.has_next():
      row = result.get_next()
      print(row)

  # Check Movies
  print("\nMovies in database:")
  result = conn.execute("MATCH (m:Movie) RETURN m.name")
  while result.has_next():
      row = result.get_next()
      print(row)

  # Check Relationships
  print("\nActing relationships:")
  result = conn.execute("MATCH (p:Person)-[:ActedIn]->(m:Movie) RETURN p.name, m.name")
  while result.has_next():
      row = result.get_next()
      print(row)

except Exception as e:
  print(f"Error occurred: {e}")


Verifying data:

People in database:
['Al Pacino', '1940-04-25']
['Robert De Niro', '1943-08-17']

Movies in database:
['The Godfather']
['The Godfather: Part II']
['The Godfather Coda: The Death of Michael Corleone']

Acting relationships:
['Al Pacino', 'The Godfather']
['Al Pacino', 'The Godfather: Part II']
['Al Pacino', 'The Godfather Coda: The Death of Michael Corleone']
['Robert De Niro', 'The Godfather: Part II']


In [ ]:
# Recreate the graph and chain with the verified database
graph = KuzuGraph(db)
chain = KuzuQAChain.from_llm(
  llm,
  graph=graph,
  verbose=True,
  allow_dangerous_requests=True
)

# Try some example questions
try:
  print("\nQuery 1: Movies Al Pacino acted in:")
  response = chain.run("What movies did Al Pacino act in?")
  print(response)

  print("\nQuery 2: Actors in Godfather Part II:")
  response = chain.run("Who acted in 'The Godfather: Part II'?")
  print(response)

except Exception as e:
  print(f"Error in chain: {e}")
  # If we get an error, let's try a direct query
  print("\nTrying direct query:")
  result = conn.execute("""
      MATCH (p:Person)-[:ActedIn]->(m:Movie)
      WHERE p.name = 'Al Pacino'
      RETURN m.name
  """)
  while result.has_next():
      print(result.get_next())


Query 1: Movies Al Pacino acted in:


> Entering new KuzuQAChain chain...
Generated Cypher:
(:Person{name: "Al Pacino"})-[:ActedIn]->(:Movie)
Error in chain: Parser exception: mismatched input '(' expecting {ALTER, ATTACH, BEGIN, CALL, CHECKPOINT, COMMENT, COMMIT, COPY, CREATE, DELETE, DETACH, DROP, EXPLAIN, EXPORT, IMPORT, INSTALL, LOAD, MATCH, MERGE, OPTIONAL, PROFILE, PROJECT, RETURN, ROLLBACK, SET, UNWIND, USE, WITH, SP} (line: 1, offset: 0)
"(:Person{name: "Al Pacino"})-[:ActedIn]->(:Movie)"
 ^

Trying direct query:
['The Godfather']
['The Godfather: Part II']
['The Godfather Coda: The Death of Michael Corleone']


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Create a custom prompt template that generates Kuzu-compatible queries
kuzu_prompt = PromptTemplate(
  input_variables=["question"],
  template="""Given this question: {question}
  Generate a Kuzu query using this syntax:
  MATCH (node_label:Type) WHERE condition RETURN values

  For example:
  - For finding movies by actor: MATCH (p:Person)-[:ActedIn]->(m:Movie) WHERE p.name = 'Actor Name' RETURN m.name
  - For finding actors in a movie: MATCH (p:Person)-[:ActedIn]->(m:Movie) WHERE m.name = 'Movie Name' RETURN p.name

  Generate only the query, no explanation:"""
)

# Create a chain that generates Kuzu queries
query_chain = LLMChain(llm=llm, prompt=kuzu_prompt)

def execute_query(question):
  try:
      # Generate the query
      query = query_chain.run(question)
      print(f"Generated query: {query}")

      # Execute the query
      result = conn.execute(query)

      # Collect results
      answers = []
      while result.has_next():
          answers.append(result.get_next())

      return answers
  except Exception as e:
      print(f"Error: {e}")
      return None

# Test the new approach
questions = [
  "What movies did Al Pacino act in?",
  "Who acted in 'The Godfather: Part II'?",
  "When was Al Pacino born?"
]

for question in questions:
  print(f"\nQuestion: {question}")
  results = execute_query(question)
  if results:
      print("Results:")
      for result in results:
          print(result)

<ipython-input-27-3dbf43c2b0e5>:19: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  query_chain = LLMChain(llm=llm, prompt=kuzu_prompt)



Question: What movies did Al Pacino act in?
Generated query: MATCH (m:Movie)-[:ActedIn]->(p:Person) WHERE p.name = 'Al Pacino' RETURN m.name

Question: Who acted in 'The Godfather: Part II'?
Generated query: MATCH (p:Person)-[:ActedIn]->(m:Movie) WHERE m.title = 'The Godfather: Part II' RETURN p.name
Error: Binder exception: Cannot find property title for m.

Question: When was Al Pacino born?
Generated query: MATCH (p:Person) WHERE p.name = 'Al Pacino' RETURN p.birthDate
Results:
['1940-04-25']


In [ ]:
from langchain.prompts import PromptTemplate
from warnings import filterwarnings


# Updated prompt template with more specific examples and correct property names
kuzu_prompt = PromptTemplate(
  input_variables=["question"],
  template="""Given this question: {question}
  Generate a Kuzu query using exactly this syntax and these property names:
  - Person properties: name, birthDate
  - Movie properties: name (not title)
  - Relationship: ActedIn (from Person TO Movie)

  Examples:
  - For movies by actor: MATCH (p:Person)-[:ActedIn]->(m:Movie) WHERE p.name = 'Actor Name' RETURN m.name
  - For actors in movie: MATCH (p:Person)-[:ActedIn]->(m:Movie) WHERE m.name = 'Movie Name' RETURN p.name
  - For birth date: MATCH (p:Person) WHERE p.name = 'Actor Name' RETURN p.birthDate

  Generate only the query, no explanation:"""
)

query_chain = LLMChain(llm=llm, prompt=kuzu_prompt)

def execute_query(question):
  try:
      # Generate the query
      query = query_chain.run(question).strip()
      print(f"Generated query: {query}")

      # Execute the query
      result = conn.execute(query)

      # Collect and format results
      answers = []
      while result.has_next():
          answers.append(result.get_next())

      return answers
  except Exception as e:
      print(f"Error: {e}")
      return None

# Test with the same questions
questions = [
  "What movies did Al Pacino act in?",
  "Who acted in 'The Godfather: Part II'?",
  "When was Al Pacino born?"
]

for question in questions:
  print(f"\nQuestion: {question}")
  results = execute_query(question)
  if results:
      print("Results:")
      for result in results:
          print(result)


Question: What movies did Al Pacino act in?
Generated query: MATCH (p:Person)-[:ActedIn]->(m:Movie) WHERE p.name = 'Al Pacino' RETURN m.name
Results:
['The Godfather']
['The Godfather: Part II']
['The Godfather Coda: The Death of Michael Corleone']

Question: Who acted in 'The Godfather: Part II'?
Generated query: MATCH (p:Person)-[:ActedIn]->(m:Movie) WHERE m.name = 'The Godfather: Part II' RETURN p.name
Results:
['Al Pacino']
['Robert De Niro']

Question: When was Al Pacino born?
Generated query: MATCH (p:Person) WHERE p.name = 'Al Pacino' RETURN p.birthDate
Results:
['1940-04-25']


In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Create the prompt template
kuzu_prompt = PromptTemplate(
  input_variables=["question"],
  template="""Given this question: {question}
  Generate a Kuzu query using exactly this syntax and these property names:
  - Person properties: name, birthDate
  - Movie properties: name (not title)
  - Relationship: ActedIn (from Person TO Movie)

  Examples:
  - For movies by actor: MATCH (p:Person)-[:ActedIn]->(m:Movie) WHERE p.name = 'Actor Name' RETURN m.name
  - For actors in movie: MATCH (p:Person)-[:ActedIn]->(m:Movie) WHERE m.name = 'Movie Name' RETURN p.name
  - For birth date: MATCH (p:Person) WHERE p.name = 'Actor Name' RETURN p.birthDate

  Generate only the query, no explanation:"""
)

# Create the chain using the new syntax
chain = {"question": RunnablePassthrough()} | kuzu_prompt | llm

def execute_query(question):
  try:
      # Generate the query
      query = chain.invoke(question).strip()
      print(f"Generated query: {query}")

      # Execute the query
      result = conn.execute(query)

      # Collect and format results
      answers = []
      while result.has_next():
          answers.append(result.get_next())

      return answers
  except Exception as e:
      print(f"Error: {e}")
      return None

# Let's try some more complex questions
questions = [
  "What movies did Al Pacino act in?",
  "Who acted in 'The Godfather: Part II'?",
  "When was Al Pacino born?",
  "Which actors appeared in movies released after 1974?"
]

for question in questions:
  print(f"\nQuestion: {question}")
  results = execute_query(question)
  if results:
      print("Results:")
      for result in results:
          print(result)


Question: What movies did Al Pacino act in?
Generated query: MATCH (p:Person)-[:ActedIn]->(m:Movie) WHERE p.name = 'Al Pacino' RETURN m.name
Results:
['The Godfather']
['The Godfather: Part II']
['The Godfather Coda: The Death of Michael Corleone']

Question: Who acted in 'The Godfather: Part II'?
Generated query: MATCH (p:Person)-[:ActedIn]->(m:Movie) WHERE m.name = 'The Godfather: Part II' RETURN p.name
Results:
['Al Pacino']
['Robert De Niro']

Question: When was Al Pacino born?
Generated query: MATCH (p:Person) WHERE p.name = 'Al Pacino' RETURN p.birthDate
Results:
['1940-04-25']

Question: Which actors appeared in movies released after 1974?
Generated query: MATCH (p:Person)-[:ActedIn]->(m:Movie) WHERE p.birthDate > 1974 RETURN p.name
